<a href="https://colab.research.google.com/github/hypro2/hands-on-LLM-from-colab/blob/main/embeddinggemma.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install git+https://github.com/huggingface/transformers@v4.56.0-Embedding-Gemma-preview
!pip install sentence-transformers>=5.0.0


In [1]:
import logging
import traceback

from datasets import load_dataset
from sentence_transformers import (
    SentenceTransformer,
    SentenceTransformerModelCardData,
    SentenceTransformerTrainer,
    SentenceTransformerTrainingArguments,
)
from sentence_transformers.evaluation import InformationRetrievalEvaluator
from sentence_transformers.losses import CachedMultipleNegativesRankingLoss
from sentence_transformers.training_args import BatchSamplers

# 로그 레벨을 INFO로 설정하여 더 자세한 정보를 얻습니다.
logging.basicConfig(format="%(asctime)s - %(message)s", datefmt="%Y-%m-%d %H:%M:%S", level=logging.INFO)

# 1. 미세 조정을 위한 모델 및 (선택 사항) 모델 카드 데이터를 불러옵니다.
model = SentenceTransformer(
    "google/embeddinggemma-300m",
    model_card_data=SentenceTransformerModelCardData(
        language="en",
        license="apache-2.0",
        model_name="EmbeddingGemma-300m train",
    ),
)

In [2]:
import pandas as pd
from datasets import Dataset
from sentence_transformers import (
    InputExample,
    SentenceTransformer,
    SentenceTransformerTrainer,
    SentenceTransformerTrainingArguments
)
from sentence_transformers.losses import MultipleNegativesRankingLoss
from sentence_transformers.evaluation import InformationRetrievalEvaluator
from sklearn.model_selection import train_test_split

# 2. 파인튜닝에 사용할 데이터셋 로드
try:
    full_dataset = load_dataset("json", data_files="hard_train_wipsglobal_search_dataset.json", split="train")
except FileNotFoundError:
    logging.error("데이터 파일 'hard_train_wipsglobal_search_dataset.json'을 찾을 수 없습니다.")
    exit()


# 3. 데이터셋을 학습(80%), 검증(10%), 테스트(10%)용으로 분할
# 먼저 학습용(80%)과 나머지(20%)로 분할
train_test_split = full_dataset.train_test_split(test_size=0.2, seed=42) # 재현성을 위해 seed 추가
train_dataset = train_test_split["train"]
eval_test_dataset = train_test_split["test"] # 검증+테스트 데이터

# 나머지(20%)를 다시 반으로 나누어 검증용(10%)과 테스트용(10%)으로 분할
eval_test_split = eval_test_dataset.train_test_split(test_size=0.5, seed=42)
eval_dataset = eval_test_split["train"]
test_dataset = eval_test_split["test"]


logging.info(f"전체 데이터셋 크기: {len(full_dataset)}")
logging.info(f"학습 데이터셋 크기: {len(train_dataset)}")
logging.info(f"검증 데이터셋 크기: {len(eval_dataset)}")
logging.info(f"테스트 데이터셋 크기: {len(test_dataset)}")


Generating train split: 0 examples [00:00, ? examples/s]

In [3]:
train_dataset[1]

{'query': ' How does the UPC impact the enforcement of unitary patents?',
 'document': "The Unified Patent Court (UPC) is a significant development in the European patent landscape, particularly regarding the enforcement of unitary patents. Here’s how the UPC impacts this process:\n\n1. **Centralized Jurisdiction**: The UPC provides a centralized court system that has jurisdiction over the enforcement of unitary patents across participating EU member states. This means that patent holders can enforce their rights in multiple countries through a single legal proceeding, rather than having to initiate separate lawsuits in each country where they seek enforcement.\n\n2. **Efficiency and Cost-Effectiveness**: By consolidating patent litigation into one court, the UPC aims to reduce the costs and complexities associated with enforcing patent rights. This can be especially beneficial for small and medium-sized enterprises (SMEs) that may not have the resources to pursue multiple legal action

In [4]:
from sentence_transformers.evaluation import InformationRetrievalEvaluator
from sentence_transformers.losses import CachedMultipleNegativesRankingLoss
from sentence_transformers.training_args import BatchSamplers

# 4. 손실 함수를 정의합니다. CachedMultipleNegativesRankingLoss (CMNRL)는 MNRL (일명 InfoNCE)의 특별한 변형입니다.
# 이 손실 함수는 질문-답변 쌍(또는 삼중항 등)을 입력으로 받습니다.
# 배치 내의 다른 질문에 대한 답변을 오답으로 사용하여, 임베딩 공간에서 질문과 정답 사이의 거리를 줄이고
# 오답과의 거리를 늘립니다.
# (C)MNRL 손실은 더 큰 `per_device_train_batch_size`에서 더 잘 작동합니다. 이는 더 많은 배치 내 오답을 활용할 수 있기 때문입니다.
# 동시에 `mini_batch_size`는 훈련 성능에 영향을 미치지 않지만, 메모리 사용량을 제한합니다.
# 좋은 방법은 `per_device_train_batch_size`를 높게 설정하고 `mini_batch_size`는 작게 유지하는 것입니다.
loss = CachedMultipleNegativesRankingLoss(model, mini_batch_size=8)

# 5. (선택 사항) 훈련 인수를 지정합니다.
# 5. 훈련 인수를 지정합니다.
# 실행 이름과 모델 저장 경로를 3항 데이터 훈련에 맞게 변경합니다.
run_name = "embeddinggemma-300m-all-nli-triplet"

args = SentenceTransformerTrainingArguments(
    output_dir=f"models/{run_name}",
    max_steps=500,
    # num_train_epochs=1,                     # 전체 데이터셋을 1번 학습합니다. (짧은 테스트 시에는 max_steps 사용)
    per_device_train_batch_size=4,         # GPU 메모리에 따라 조절하세요. 8, 16, 32 등이 일반적입니다.
    learning_rate=2e-5,                     # fine-tuning 시 일반적으로 사용되는 학습률입니다.
    warmup_ratio=0.1,                       # 전체 훈련 스텝의 10% 동안 학습률을 서서히 증가시켜 안정적인 학습을 돕습니다.
    batch_sampler=BatchSamplers.NO_DUPLICATES, # 배치 내에 중복된 문장이 들어가지 않도록 하여 학습 효율을 높입니다.
    fp16=True,                              # NVIDIA GPU 사용 시 학습 속도를 높여줍니다. (오류 발생 시 False로 변경)
    bf16=False,                             # 최신 GPU(A100 등)에서 지원하는 경우 사용합니다.
    per_device_eval_batch_size=4,
    eval_strategy="steps",                  # 특정 스텝마다 평가를 진행합니다.
    eval_steps=100,                         # 500 스텝마다 dev 데이터셋으로 모델 성능을 평가합니다.
    save_strategy="steps",                  # 특정 스텝마다 모델을 저장합니다.
    save_steps=100,                         # 500 스텝마다 체크포인트를 저장합니다.
    save_total_limit=2,                     # 최대 2개의 체크포인트만 유지합니다. (오래된 순으로 삭제)
    logging_steps=100,                      # 100 스텝마다 훈련 손실(loss) 등의 로그를 출력합니다.
    run_name=run_name,                      # wandb.ai와 같은 실험 추적 도구에서 사용될 실행 이름입니다.
)


In [7]:
from sentence_transformers.evaluation import TripletEvaluator

# 3항 데이터셋의 각 열을 평가자에 직접 전달합니다.
print("Creating the TripletEvaluator for the dev set...")
dev_evaluator = TripletEvaluator(
    anchors=eval_dataset["query"],
    positives=eval_dataset["document"],
    negatives=eval_dataset["negative"],
    name="all-nli-dev", # 평가 결과에 표시될 이름
    show_progress_bar=True,
)

# 생성한 평가자로 모델의 성능을 측정합니다.
# 이 코드는 학습 전/후에 호출하여 성능 변화를 확인할 수 있습니다.
dev_evaluator(model)

Creating the TripletEvaluator for the dev set...


Batches:   0%|          | 0/97 [00:00<?, ?it/s]

Batches:   0%|          | 0/97 [00:00<?, ?it/s]

Batches:   0%|          | 0/97 [00:00<?, ?it/s]

{'all-nli-dev_cosine_accuracy': 0.9987063407897949}

In [8]:
# 7. 트레이너를 생성하고 훈련을 시작합니다.
trainer = SentenceTransformerTrainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    loss=loss,
    # evaluator=dev_evaluator,
)
trainer.train()


Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: hyeong9647 (hyeong9647-personal) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss,Validation Loss
100,0.012800,0.017143
200,0.012600,0.032005
300,0.004500,0.010067
400,0.002500,0.008838
500,0.003900,0.007319


TrainOutput(global_step=500, training_loss=0.007270585745573044, metrics={'train_runtime': 1495.6445, 'train_samples_per_second': 1.337, 'train_steps_per_second': 0.334, 'total_flos': 0.0, 'train_loss': 0.007270585745573044, 'epoch': 0.16170763260025872})

In [10]:
from sentence_transformers.evaluation import TripletEvaluator

# 3항 데이터셋의 각 열을 평가자에 직접 전달합니다.
print("Creating the TripletEvaluator for the dev set...")
dev_evaluator = TripletEvaluator(
    anchors=eval_dataset["query"],
    positives=eval_dataset["document"],
    negatives=eval_dataset["negative"],
    name="all-nli-dev", # 평가 결과에 표시될 이름
    show_progress_bar=True,
)

# 생성한 평가자로 모델의 성능을 측정합니다.
# 이 코드는 학습 전/후에 호출하여 성능 변화를 확인할 수 있습니다.
dev_evaluator(model)

Creating the TripletEvaluator for the dev set...


Batches:   0%|          | 0/97 [00:00<?, ?it/s]

Batches:   0%|          | 0/97 [00:00<?, ?it/s]

Batches:   0%|          | 0/97 [00:00<?, ?it/s]

{'all-nli-dev_cosine_accuracy': 0.9980595111846924}

In [9]:
# 8. 훈련된 모델을 저장합니다.
final_output_dir = f"models/{run_name}/final"
model.save_pretrained(final_output_dir)